# 🧪 Super Resolución Clásica - Técnicas en Python
Este notebook muestra implementaciones prácticas de cinco técnicas clásicas de super resolución: 
1. Interpolación (bilineal y bicúbica)
2. Multi-frame Super Resolution (MFSR)
3. Sparse Representation
4. Edge-Directed Interpolation
5. Neighbor Embedding


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, img_as_float
from skimage.transform import warp, AffineTransform, resize
from skimage.filters import sobel
from sklearn.decomposition import SparseCoder
from sklearn.feature_extraction.image import extract_patches_2d, reconstruct_from_patches_2d
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors


## 1. Interpolación bilineal y bicúbica

In [ ]:
# Cargar imagen
img_path = "imagen_lr.jpg"  # Reemplaza con tu imagen
img = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Interpolaciones
bilinear = cv2.resize(img_rgb, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
bicubic = cv2.resize(img_rgb, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

plt.figure(figsize=(12,4))
plt.subplot(1, 3, 1); plt.title("Original"); plt.imshow(img_rgb)
plt.subplot(1, 3, 2); plt.title("Bilineal"); plt.imshow(bilinear)
plt.subplot(1, 3, 3); plt.title("Bicúbica"); plt.imshow(bicubic)
plt.show()

## 2. Multi-frame Super Resolution (simulado)

In [ ]:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_float = img_as_float(img_gray)

# Simular desplazamientos
def downsample(img):
    return cv2.resize(img, (0, 0), fx=0.5, fy=0.5)

shifts = [(0, 0), (1, 1), (-1, -1)]
imgs_lr = []
for dx, dy in shifts:
    tform = AffineTransform(translation=(dx, dy))
    warped = warp(img_float, tform.inverse)
    imgs_lr.append(downsample(warped))

imgs_up = [cv2.resize(i, (img_float.shape[1], img_float.shape[0]), interpolation=cv2.INTER_CUBIC)
           for i in imgs_lr]
sr_avg = np.mean(imgs_up, axis=0)

plt.imshow(sr_avg, cmap='gray'); plt.title("Multi-frame SR (Media)")
plt.axis('off')
plt.show()

## 3. Sparse Representation (simulado)

In [ ]:
np.random.seed(0)
D_LR = normalize(np.random.rand(64, 256))
D_HR = normalize(np.random.rand(64, 256))

patches = extract_patches_2d(img_gray, (8, 8), max_patches=1000)
patches = patches.reshape((patches.shape[0], -1))

coder = SparseCoder(dictionary=D_LR, transform_n_nonzero_coefs=5, transform_algorithm='omp')
alpha = coder.transform(patches)
recon_patches = np.dot(alpha, D_HR.T)
recon_patches = recon_patches.reshape((-1, 8, 8))
output_sparse = reconstruct_from_patches_2d(recon_patches, img_gray.shape)

plt.imshow(output_sparse, cmap='gray'); plt.title("Sparse Coding SR")
plt.axis('off')
plt.show()

## 4. Edge-Directed Interpolation (simplificado)

In [ ]:
edge_map = sobel(img_float)
upsampled = resize(img_float, (img_float.shape[0]*2, img_float.shape[1]*2), order=1)
sharp = upsampled + 0.3 * resize(edge_map, upsampled.shape)

plt.imshow(sharp, cmap='gray'); plt.title("Edge-Directed Interpolation")
plt.axis('off')
plt.show()

## 5. Neighbor Embedding (simulado)

In [ ]:
num_patches = 500
D_LR = np.random.rand(num_patches, 64)
D_HR = np.random.rand(num_patches, 64)
patch_test = np.random.rand(64).reshape(1, -1)

# Vecinos más cercanos
nn = NearestNeighbors(n_neighbors=5).fit(D_LR)
_, indices = nn.kneighbors(patch_test)
neighs = D_LR[indices[0]]
dists = np.linalg.norm(neighs - patch_test, axis=1)
weights = 1 / (dists + 1e-6)
weights /= weights.sum()
hr_patch = np.sum(weights[:, None] * D_HR[indices[0]], axis=0)

plt.plot(hr_patch)
plt.title("Reconstrucción de parche HR")
plt.grid(True)
plt.show()